# How to build the circuit of U-layer + v-layer 

## Prepare environment and  define parameters 

In [ ]:
is_colab = False
import sys
if is_colab:
    !pip install -q torch==1.8.1
    !pip install -q torchvision==0.4.0
    !pip install -q qiskit==0.20.0
    !pip install qfnn
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w9VRv0iVfsH20Kb_MkF3yFhFeiYDVy5n' -O model.tar.gz
    !tar zxvf /content/model.tar.gz
    
import torch
from qiskit import  QuantumCircuit, ClassicalRegister
import numpy as np
import functools
from qfnn.qf_fb.q_output import fire_ibmq,analyze,add_measure
from qfnn.qf_circ.u_lyr_circ import U_LYR_Circ
from qfnn.qf_circ.v_lyr_circ import V_LYR_Circ
from qfnn.qf_net.utils import binarize
from qfnn.qf_fb.c_input import load_data,to_quantum_matrix
from qfnn.qf_fb.c_qf_mixer import Net
from qfnn.qf_fb.c_input import ToQuantumData
print = functools.partial(print, flush=True)

################ Zhirui on 12-30-2020 ################
# path
# remember to change the path on your computer
######################################################
if is_colab:
    data_path = '/content/data' #mnist  path
    resume_path = '/content/model/u4_v2/model_best.tar' #model path
else:
    data_path = '/home/hzr/Software/quantum/qc_mnist/pytorch/data' #mnist  path
    resume_path = '/home/hzr/Software/quantum/QuantumFlow_Tutorial/model/u4_v2/model_best.tar' #model path

In [ ]:

################ Zhirui on 12-30-2020 ################
# Parameters of settings
######################################################
interest_num = [3,6]
img_size = 4
batch_size = 32# how many samples per batch to load
inference_batch_size = 1
isppd = False #is prepared data


## Load data

In [ ]:
################ Zhirui on 12-30-2020 ################
# load data.
######################################################

train_loader, test_loader = load_data(interest_num,data_path,isppd,img_size,batch_size,inference_batch_size,False)
for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix = to_quantum_matrix(data)
    break

## Inference

### load model parameter

In [ ]:


# Model initialization

checkpoint = torch.load(resume_path, map_location='cpu')
weight = checkpoint['state_dict']['fc0.weight']
theta = checkpoint['state_dict']['fc2.theta']


### build the network : the first layer :u-layer

In [ ]:
################ Weiwen on 12-30-2020 ################
# Generate the circuit of u-layer
######################################################

#define your input and output number
u_layer = U_LYR_Circ(4,4) 
#init circuit
circuit = QuantumCircuit()
#add input qubit to your circuit
inps = u_layer.add_input_qubits(circuit)

#add auxiliary qubit to your circuit
aux =u_layer.add_aux(circuit)

#add output qubit to your circuit
u_layer_out_qubits = u_layer.add_out_qubits(circuit)

#add u-layer to your circuit
u_layer.forward(circuit,binarize(weight),inps,u_layer_out_qubits,quantum_matrix,aux)

#add measurement
circuit.barrier()
add_measure(circuit,u_layer_out_qubits,'reg')

#show your circuit
circuit.draw('text',fold=300)



### u-layer simulation

In [ ]:
################ hzr on 12-30-2020 ################
# u-layer simulation
######################################################

qc_shots=10000
u_layer_counts = fire_ibmq(circuit,qc_shots,True)
print(u_layer_counts)
(u_layer_counts,u_layer_bits) = analyze(u_layer_counts)



### build the network : the second layer :v-layer

In [ ]:

import math
#init new circuit
circuit2 = QuantumCircuit()
vqc = V_LYR_Circ(4,1)
v_inputs = vqc.add_input_qubits(circuit2)

#init state by u-layer-measurement (angle encoding)
for b in range(u_layer_bits):
    prob =float(u_layer_counts[b])/qc_shots
    circuit2.ry(2*math.asin(math.sqrt(prob)),b)

#add vqc to the circuit
vqc.forward(circuit2,v_inputs,'v10',np.array(theta,dtype=np.double))   


#add measurement to the circuit2
circuit2.barrier()
add_measure(circuit2,[v_inputs[0][3],v_inputs[0][2]],'reg_v')
circuit2.draw('text',300)

### v-layer simulation

In [ ]:

opt_counts = fire_ibmq(circuit2,qc_shots,True)
print(opt_counts)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]

for b in range(2):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)

print("Result of u+v:",opt_class_prob)
print("Prediction class: {}".format(opt_class_prob.index(max(opt_class_prob))))
print("Target class: {}".format(target[0]))
if opt_class_prob.index(max(opt_class_prob))==target[0]:
    print("Correct prediction")
else:
    print("Incorrect prediction")
print("="*30)


## classical inference

In [ ]:

neural_in_layers = 'u:4,p2a:16,v:2'
layers = []
for item1 in neural_in_layers.split(","):
    x= item1.split(":")
    layer =[]
    layer.append(x[0].strip())
    layer.append(int(x[1].strip()))
    layers.append(layer)

model = Net(img_size,layers,False,False)
model.load_state_dict(checkpoint["state_dict"])
to_quantum_data = ToQuantumData(img_size)
output_data = to_quantum_data(data)
output = model.forward(output_data,False)
print("classical inference result :",output)